#  쿼리 확장 (Query Expansion)

--- 

## 1. 개념 이해

### 1.1 쿼리 확장이란?

- 쿼리 확장(Query Expansion)은 사용자의 원본 질문을 보다 효과적인 검색을 위해 변형하거나 확장하는 기법입니다. 
- RAG 시스템에서 검색의 품질은 최종 답변의 품질을 좌우하는 핵심 요소이므로, 적절한 쿼리 확장 기법을 통해 검색 성능을 크게 향상시킬 수 있습니다.

### 1.2 쿼리 확장의 필요성

#### 🔍 **검색의 한계**
- **어휘 불일치 문제**: 사용자 질문과 문서 간 용어 차이
- **모호한 질문**: 불명확하거나 맥락이 부족한 쿼리
- **복잡한 질문**: 여러 하위 질문을 포함하는 복합 질문
- **의도 파악 어려움**: 사용자의 실제 의도와 표면적 질문의 차이

#### 🎯 **해결 방향**
- **의미론적 확장**: 동의어, 관련 개념 추가
- **구조적 분해**: 복잡한 질문을 단순한 하위 질문으로 분해
- **맥락적 확장**: 배경 지식과 일반적 개념 추가
- **가상 문서 생성**: 이상적 답변을 통한 검색 개선

### 1.3 쿼리 확장 방법론 분류

| 방법론 | 핵심 아이디어 | 장점 | 단점 | 적용 상황 |
|--------|---------------|------|------|-----------|
| **Query Reformulation** | LLM으로 질문 재작성 | 구현 간단, 즉시 적용 | 단일 변형만 생성 | 일반적인 질문 개선 |
| **Multi Query** | 다양한 관점의 질문 생성 | 검색 다양성 증가 | 계산 비용 증가 | 모호한 질문 처리 |
| **Decomposition** | 복잡한 질문을 하위 질문으로 분해 | 체계적 접근 | 분해 정확도 의존 | 복합적 질문 |
| **Step-Back Prompting** | 일반적 맥락에서 구체적 답변으로 | 포괄적 이해 | 추가 검색 필요 | 전문적/복잡한 질문 |
| **HyDE** | 가상 답변 문서 생성 | 의미적 정렬 우수 | 환각 위험 | Zero-shot 상황 |

---

## 2. 환경 설정

### 2.1 필수 라이브러리 설치

```bash
# 기본 LangChain 라이브러리
pip install langchain langchain-community langchain-openai
pip install langchain-chroma

# 추가 도구
pip install python-dotenv
pip install matplotlib seaborn pandas numpy

# 선택적 라이브러리
pip install tiktoken  # 토큰 계산
```

### 2.2 환경 변수 설정


In [1]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

True

In [2]:
# Langsmith tracing 여부를 확인 (true: langsmith 추적 활성화, false: langsmith 추적 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

true


### 2.3 기본 라이브러리 Import

In [3]:
import os
import json
from pprint import pprint
from typing import List, Dict, Any
import pandas as pd
import numpy as np

# LangChain 핵심
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, BaseOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 검색 및 벡터 저장
from langchain_chroma import Chroma
from langchain.retrievers.multi_query import MultiQueryRetriever

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

### 2.4 벡터 저장소 초기화

In [4]:
def initialize_vector_store(embeddings = OpenAIEmbeddings(model="text-embedding-3-small"), collection_name="hybrid_search_db", persist_directory = "./local_chroma_db"):
    """
    기존 벡터 저장소를 로드하거나 새로 생성
    
    Returns:
        Chroma: 벡터 저장소 객체
    """
    try:
        
        # 기존 벡터 저장소 로드 시도
        vector_store = Chroma(
            collection_name=collection_name,
            embedding_function=embeddings,
            persist_directory=persist_directory,
        )
        
        doc_count = vector_store._collection.count()
        if doc_count > 0:
            print(f"✅ 기존 벡터 저장소 로드: {doc_count}개 문서")
            return vector_store
        else:
            print("⚠️ 빈 벡터 저장소입니다. 데이터를 추가해주세요.")
            return vector_store
            
    except Exception as e:
        print(f"❌ 벡터 저장소 로드 실패: {e}")
        return None

# 벡터 저장소 초기화
vector_store = initialize_vector_store()

if vector_store:
    # 기본 검색기 생성
    base_retriever = vector_store.as_retriever(search_kwargs={"k": 5})
    print("✅ 기본 검색기 생성 완료")

✅ 기존 벡터 저장소 로드: 39개 문서
✅ 기본 검색기 생성 완료


In [5]:
# 기본 retriever 테스트 

query = "리비안의 사업 경쟁력은 어디서 나오나요?"
retrieved_docs = base_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

<Document>
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J. 스캐린지
- **본사:** 미국 캘리포니아 주 어바인
- **서비스 지역:** 북미
- **주요 인물:** R. J. 스캐린지 (CEO)
- **제품:** 전기 자동차, 배터리
- **생산량 (2023):** 57,232대
- **서비스:** 전기 자동차 충전, 자동차 보험
- **수익 (2023):** 44억 3천만 미국 달러
- **순이익 (2023):** -54억 미국 달러
- **총 자산 (2023):** 168억 미국 달러
</Document>
<Source>이 문서는 '리비안(rivian)'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
**시설**

- **Irvine, California:** 차량 엔지니어링 및 설계에 중점을 둔 본사.
- **Normal, Illinois:** 차량 부품을 생산하고 조립을 수행하는 제조 공장.
- **Plymouth, Michigan:** 차량 엔지니어링, 프로토타입 제작, 공급망 및 회계에 중점을 둡니다.
- **Palo Alto, California:** 소프트웨어 개발 및 엔지니어링에 중점을 둡니다.
- Carson, California 및 Woking, England에 추가 사무실이 있습니다.
- 애틀랜타 동쪽에 있는 새로운 50억 달러 규모의 배터리 및 조립 공장은 보류 중입니다.

**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.
</Document>
<Source>이 문서는 '리비안(rivian)'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.

| 연도 | 수익 (백만 USD) 

---

## 3. Query Reformulation

### 3.1 개념과 특징

Query Reformulation은 가장 기본적인 쿼리 확장 기법으로, LLM을 사용하여 원본 질문을 더 명확하고 검색에 적합한 형태로 재작성하는 방법입니다.

#### 🎯 **핵심 특징**
- 동의어 확장 및 키워드 추가
- 모호한 표현을 구체적으로 변환
- 검색 친화적 용어로 재구성
- 단일 개선된 쿼리 생성

<div align="center">
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/query_rewrite.png" alt="Query Reformulation" width="700">
<br><i>Query Reformulation 프로세스</i>
</div>

### 3.2 구현

#### 3.2.1 기본 Reformulation

In [6]:
def create_reformulation_chain():
    """
    기본적인 쿼리 재작성 체인 생성
    Returns:
        체인 객체
    """
    
    # 프롬프트 템플릿 정의
    reformulation_template = ChatPromptTemplate.from_messages([
        ("system", """당신은 검색 쿼리 최적화 전문가입니다.
주어진 질문을 검색 성능 향상을 위해 재작성해주세요.
다음 원칙을 따라 재작성하세요:
1. 핵심 키워드 유지 및 강화
2. 동의어나 관련 용어 추가
3. 모호한 표현을 구체적으로 변경
4. 검색 엔진이 이해하기 쉬운 형태로 변환
5. 원래 질문의 의도는 반드시 보존

재작성된 질문만 응답하세요."""),
        ("user", "원본 질문: {question}")
    ])
    
    # 체인 구성
    llm = ChatOpenAI(model='gpt-4.1-mini', temperature=0.1)
    
    return reformulation_template | llm | StrOutputParser()

In [7]:
# 리포뮬레이션된 쿼리로 검색

reformulation_chain = create_reformulation_chain()
reformulated_query = reformulation_chain.invoke({"question": query})

print(reformulated_query)

retrieved_docs = base_retriever.invoke(reformulated_query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

리비안 전기차 기업의 사업 경쟁력 원천과 주요 강점은 무엇인가요?
<Document>
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J. 스캐린지
- **본사:** 미국 캘리포니아 주 어바인
- **서비스 지역:** 북미
- **주요 인물:** R. J. 스캐린지 (CEO)
- **제품:** 전기 자동차, 배터리
- **생산량 (2023):** 57,232대
- **서비스:** 전기 자동차 충전, 자동차 보험
- **수익 (2023):** 44억 3천만 미국 달러
- **순이익 (2023):** -54억 미국 달러
- **총 자산 (2023):** 168억 미국 달러
</Document>
<Source>이 문서는 '리비안(rivian)'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
**시설**

- **Irvine, California:** 차량 엔지니어링 및 설계에 중점을 둔 본사.
- **Normal, Illinois:** 차량 부품을 생산하고 조립을 수행하는 제조 공장.
- **Plymouth, Michigan:** 차량 엔지니어링, 프로토타입 제작, 공급망 및 회계에 중점을 둡니다.
- **Palo Alto, California:** 소프트웨어 개발 및 엔지니어링에 중점을 둡니다.
- Carson, California 및 Woking, England에 추가 사무실이 있습니다.
- 애틀랜타 동쪽에 있는 새로운 50억 달러 규모의 배터리 및 조립 공장은 보류 중입니다.

**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.
</Document>
<Source>이 문서는 '리비안(rivian)'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
- R1T 배송은 2021년 9월에 시작되어 Rivian은 완전

#### 3.2.2 도메인 특화 Reformulation

In [8]:
def create_domain_specific_reformulation_chain(domain="technology"):
    """
    도메인 특화 쿼리 재작성 체인
    Args:
        domain (str): 도메인 분야
    Returns:
        체인 객체
    """
    
    # 도메인별 시스템 메시지 정의
    domain_system_messages = {
        "technology": """당신은 기술 분야 검색 전문가입니다. 기술 문서에서 관련 정보를 찾기 위한 최적의 검색 쿼리로 재작성해주세요.

기술 분야 재작성 원칙:
- 기술 용어의 정확성 유지
- 관련 기술 스택이나 프레임워크 포함
- 구체적인 기술적 맥락 추가
- 전문 용어와 일반 용어 조합

재작성된 기술 검색 쿼리만 응답하세요.""",
        
        "business": """당신은 비즈니스 분석 전문가입니다. 비즈니스 문서에서 관련 정보를 찾기 위한 최적의 검색 쿼리로 재작성해주세요.

비즈니스 분야 재작성 원칙:
- 비즈니스 핵심 지표 및 용어 포함
- 시장, 경쟁, 전략 관점 추가
- 산업별 특수 용어 활용
- 정량적 및 정성적 측면 고려

재작성된 비즈니스 검색 쿼리만 응답하세요.""",
        
        "academic": """당신은 학술 연구 전문가입니다. 학술 문헌에서 관련 정보를 찾기 위한 최적의 검색 쿼리로 재작성해주세요.

학술 분야 재작성 원칙:
- 학술적 용어와 개념 사용
- 연구 방법론 및 이론적 배경 고려
- 관련 연구 분야 및 키워드 포함
- 정확하고 객관적인 표현 사용

재작성된 학술 검색 쿼리만 응답하세요."""
    }
    
    # 선택된 도메인의 시스템 메시지 가져오기 (기본값: technology)
    system_message = domain_system_messages.get(domain, domain_system_messages["technology"])
    
    # ChatPromptTemplate를 사용하여 시스템 메시지와 유저 메시지로 구성
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("user", "원본 질문: {question}")
    ])
    
    # 체인 구성
    llm = ChatOpenAI(model='gpt-4.1-mini', temperature=0.1)
    
    return prompt_template | llm | StrOutputParser()

In [9]:
test_question = "전기차 회사의 성장 전략은?"

domains = ["technology", "business", "academic"]

print("🏢 도메인별 Query Reformulation")
print("=" * 80)
print(f"원본 질문: {test_question}\n")

for domain in domains:
    chain = create_domain_specific_reformulation_chain(domain)
    reformulated = chain.invoke({"question": test_question})
    
    print(f"[{domain.upper()}] {reformulated}")
    print("-" * 50)

🏢 도메인별 Query Reformulation
원본 질문: 전기차 회사의 성장 전략은?

[TECHNOLOGY] 전기차 산업 성장 전략 사례 및 전기차 제조사 비즈니스 모델 분석
--------------------------------------------------
[BUSINESS] 전기차 산업 성장 전략, 시장 진입 및 확장 계획, 경쟁사 분석, 신기술 도입 및 R&D 투자, 고객 확보 및 유지 전략, 공급망 최적화, 정부 정책 및 규제 영향, 전기차 판매 및 매출 성장 지표
--------------------------------------------------
[ACADEMIC] 전기차 산업에서 기업 성장 전략 분석: 혁신 경영, 시장 진입 전략, 기술 개발 및 경쟁 우위 확보 방안 연구
--------------------------------------------------


### 3.3 LCEL을 활용한 통합 검색기

In [10]:
def create_reformulation_retriever(vector_store, domain="general"):
    """
    재작성 기능이 통합된 검색기 생성
    
    Args:
        vector_store: 벡터 저장소
        domain (str): 도메인 설정
    
    Returns:
        통합 검색기
    """
    # 재작성 체인
    if domain == "general":
        reformulation_chain = create_reformulation_chain()
    else:
        reformulation_chain = create_domain_specific_reformulation_chain(domain)
    
    # 기본 검색기
    base_retriever = vector_store.as_retriever(search_kwargs={"k": 5})
    
    # 통합 체인: 재작성 -> 검색
    integrated_retriever = reformulation_chain | base_retriever
    
    return integrated_retriever

In [11]:
# 통합 검색기 생성
reformulation_retriever = create_reformulation_retriever(vector_store, "business")

test_query = "리비안의 경쟁 우위는?"
results = reformulation_retriever.invoke({"question": test_query})


print("🔗 통합 검색기 결과:")
for i, doc in enumerate(results, 1):
    print(f"[{i}] {doc.page_content[:100]}...")
    print(f"    출처: {doc.metadata.get('source', 'N/A')}")
    print("=" * 50)

🔗 통합 검색기 결과:
[1] <Document>
## 비즈니스 전략

Tesla의 전략은 배터리 비용을 줄이기 위해 고가, 소량 차량으로 시작한 다음 더 저렴하고 대량 차량을 제공하는 것입니다. Tesla는 ...
    출처: data\테슬라_KR.md
[2] <Document>
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J...
    출처: data\리비안_KR.md
[3] <Document>
- R1T 배송은 2021년 9월에 시작되어 Rivian은 완전 전기 픽업을 소비자 시장에 출시한 최초의 자동차 제조업체가 됨.
- 2021년 11월, Rivi...
    출처: data\리비안_KR.md
[4] <Document>
**Volkswagen과의 파트너십 (2024)**

- 2024년 6월, Volkswagen Group은 전기 아키텍처 및 소프트웨어 기술 개발을 목표로 Ri...
    출처: data\리비안_KR.md
[5] <Document>
- 2022년 10월, 느슨한 토크 볼트로 인해 13,000대의 차량을 자발적으로 리콜.
- 2023년 6월, Tesla의 북미 충전 시스템(NACS) 채택을 ...
    출처: data\리비안_KR.md


---

## 4. Multi Query

### 4.1 개념과 특징

Multi Query는 하나의 질문에 대해 다양한 관점과 표현으로 여러 개의 쿼리를 생성하여 검색 다양성과 포괄성을 높이는 기법입니다.

#### 🎯 **핵심 특징**
- 다양한 관점에서 질문 재구성
- 검색 결과의 다양성 증가
- 누락될 수 있는 관련 문서 발견
- LLM의 창의적 질문 생성 능력 활용

<div align="center">
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/multi-query.png" alt="Multi Query" width="700">
<br><i>Multi Query 프로세스</i>
</div>

### 4.2 기본 구현

#### 4.2.1 LangChain MultiQueryRetriever 사용

- https://python.langchain.com/docs/how_to/MultiQueryRetriever/


In [12]:
def create_basic_multi_query_retriever(vector_store):
    """
    기본 MultiQueryRetriever 생성
    
    Args:
        vector_store: 벡터 저장소
    
    Returns:
        MultiQueryRetriever 객체
    """
    # LLM 초기화
    llm = ChatOpenAI(
        model='gpt-4.1-mini',
        temperature=0.7,  # 다양성을 위해 높은 temperature
    )
    
    # 기본 검색기
    base_retriever = vector_store.as_retriever(search_kwargs={"k": 3})
    
    # MultiQueryRetriever 생성
    multi_query_retriever = MultiQueryRetriever.from_llm(
        retriever=base_retriever,
        llm=llm
    )
    
    return multi_query_retriever

In [13]:
# Multi Query 데모

basic_multi_retriever = create_basic_multi_query_retriever(vector_store)

test_questions = [
    "리비안의 핵심 기술은 무엇인가요?",
    "전기차 시장의 미래 전망은?",
]

for i, question in enumerate(test_questions, 1):
    print(f"\n[{i}] 원본 질문: {question}")
    
    # Multi Query 검색 실행
    results = basic_multi_retriever.invoke(question)
    
    print(f"    검색 결과: {len(results)}개 문서 (중복 제거 후)")
    print("    상위 3개 결과:")
    
    for j, doc in enumerate(results[:3], 1):
        print(f"      [{j}] {doc.page_content[:80]}...")
        print(f"          출처: {doc.metadata.get('source', 'N/A')}")
    
    print("-" * 80)


[1] 원본 질문: 리비안의 핵심 기술은 무엇인가요?
    검색 결과: 6개 문서 (중복 제거 후)
    상위 3개 결과:
      [1] <Document>
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록l...
          출처: data\리비안_KR.md
      [2] <Document>
- 2022년 10월, 느슨한 토크 볼트로 인해 13,000대의 차량을 자발적으로 리콜.
- 2023년 6월, Tesla의 ...
          출처: data\리비안_KR.md
      [3] <Document>
**시설**

- **Irvine, California:** 차량 엔지니어링 및 설계에 중점을 둔 본사.
- **Normal...
          출처: data\리비안_KR.md
--------------------------------------------------------------------------------

[2] 원본 질문: 전기차 시장의 미래 전망은?
    검색 결과: 5개 문서 (중복 제거 후)
    상위 3개 결과:
      [1] <Document>
## 비즈니스 전략

Tesla의 전략은 배터리 비용을 줄이기 위해 고가, 소량 차량으로 시작한 다음 더 저렴하고 대량 차량...
          출처: data\테슬라_KR.md
      [2] <Document>
Tesla의 차량 생산은 2008년 Roadster로 시작하여 Model S (2012), Model X (2015), Mo...
          출처: data\테슬라_KR.md
      [3] <Document>
- R1T 배송은 2021년 9월에 시작되어 Rivian은 완전 전기 픽업을 소비자 시장에 출시한 최초의 자동차 제조업체가 ...
          출처: data\리비안_KR.md
----------------

#### 4.2.2 커스텀 Multi Query 구현

In [14]:
class CustomLineListOutputParser(BaseOutputParser[List[str]]):
    """커스텀 출력 파서: 줄 단위로 분리된 질문 리스트 생성"""
    
    def parse(self, text: str) -> List[str]:
        """텍스트를 줄 단위로 분리하여 질문 리스트 반환"""
        lines = text.strip().split("\n")
        # 빈 줄 제거하고 번호나 불릿 포인트 제거
        questions = []
        for line in lines:
            line = line.strip()
            if line:
                # 번호나 불릿 포인트 제거 (예: "1. ", "- ", "• ")
                import re
                cleaned = re.sub(r'^[\d\-\•\*\+]\s*\.?\s*', '', line)
                if cleaned:
                    questions.append(cleaned)
        return questions

In [15]:
def create_custom_multi_query_chain():
    """
    커스텀 Multi Query 체인 생성
    Returns:
        쿼리 생성 체인
    """
    # 프롬프트 템플릿 정의
    query_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 검색 쿼리 전문가입니다. 주어진 질문에 대해 다양한 관점에서 5개의 대안 질문을 생성해주세요.

생성 원칙:
1. 원본 질문의 핵심 의도를 유지하되, 다양한 표현 방식 사용
2. 서로 다른 접근 방식이나 관점 적용
3. 구체적인 질문과 일반적인 질문을 적절히 조합
4. 동의어나 관련 개념을 활용한 변형
5. 각 질문은 한 줄에 하나씩 작성

5개의 대안 질문만 응답하세요."""),
        ("user", "원본 질문: {question}")
    ])
    
    # LLM 및 파서 설정
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.7)
    output_parser = CustomLineListOutputParser()
    
    # 체인 구성
    return query_prompt | llm | output_parser

In [16]:
# 커스텀 Multi Query 체인 실행 
multi_query_chain = create_custom_multi_query_chain()

test_query = "리비안의 사업 전략은 무엇인가요?"
print(f"\n🔍 원본 질문: {test_query}")

generated_queries = multi_query_chain.invoke({"question": test_query})

print(f"    생성된 대안 질문 ({len(generated_queries)}개):")
for j, alt_query in enumerate(generated_queries, 1):
    print(f"      {j}. {alt_query}")

print("-" * 50)


🔍 원본 질문: 리비안의 사업 전략은 무엇인가요?
    생성된 대안 질문 (5개):
      1. 리비안이 추구하는 주요 사업 방향과 전략은 어떻게 되나요?
      2. 리비안의 기업 전략과 시장 진입 방식은 무엇인가요?
      3. 리비안이 전기차 산업에서 경쟁력을 확보하기 위한 전략은 무엇인가요?
      4. 리비안의 비즈니스 모델과 성장 계획에 대해 설명해 주세요.
      5. 리비안이 전기차 시장에서 성공하기 위해 채택한 핵심 전략은 무엇인가요?
--------------------------------------------------


In [17]:
# 기본 검색기
base_retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# MultiQueryRetriever 설정
advanced_retriever = MultiQueryRetriever(
    retriever=base_retriever,
    llm_chain=multi_query_chain,
    parser_key="lines"
)

results = advanced_retriever.invoke(test_query)
print(f"검색 결과: {len(results)}개 문서")
for i, doc in enumerate(results, 1):
    print(f"[{i}] {doc.page_content[:80]}... [출처: {doc.metadata.get('source', 'N/A')}]")
    print("=" * 50)

검색 결과: 7개 문서
[1] <Document>
**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.

| 연도 | 수익 (백만 US... [출처: data\리비안_KR.md]
[2] <Document>
- 2022년 10월, 느슨한 토크 볼트로 인해 13,000대의 차량을 자발적으로 리콜.
- 2023년 6월, Tesla의 ... [출처: data\리비안_KR.md]
[3] <Document>
## 비즈니스 전략

Tesla의 전략은 배터리 비용을 줄이기 위해 고가, 소량 차량으로 시작한 다음 더 저렴하고 대량 차량... [출처: data\테슬라_KR.md]
[4] <Document>
**시설**

- **Irvine, California:** 차량 엔지니어링 및 설계에 중점을 둔 본사.
- **Normal... [출처: data\리비안_KR.md]
[5] <Document>
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록l... [출처: data\리비안_KR.md]
[6] <Document>
- R1T 배송은 2021년 9월에 시작되어 Rivian은 완전 전기 픽업을 소비자 시장에 출시한 최초의 자동차 제조업체가 ... [출처: data\리비안_KR.md]
[7] <Document>
**역사**

**초창기 (2009–15):**

- 2009년 R. J. 스캐린지가 Mainstream Motors로 설립... [출처: data\리비안_KR.md]


---

## 5. Query Decomposition

### 5.1 개념과 특징

Query Decomposition은 복잡한 질문을 여러 개의 단순한 하위 질문으로 분해하여 각각을 독립적으로 처리하는 기법입니다. Least-to-Most Prompting 전략을 기반으로 합니다.

#### 🎯 **핵심 특징**
- 복잡한 다단계 질문을 단순화
- 각 하위 질문에 대한 독립적 검색
- 체계적이고 논리적인 접근
- 포괄적인 정보 수집 가능

<div align="center">
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/query_decomposition.png" alt="Query Decomposition" width="700">
<br><i>Query Decomposition 프로세스</i>
</div>

### 5.2 구현

#### 5.2.1 기본 Decomposition 체인

In [18]:
from pydantic import BaseModel, Field
from typing import List
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

class SubQuery(BaseModel):
    """하위 질문 모델"""
    sub_query: str = Field(description="분해된 하위 질문")
    reasoning: str = Field(description="이 하위 질문이 필요한 이유")

class DecompositionResult(BaseModel):
    """질문 분해 결과 모델"""
    sub_queries: List[SubQuery] = Field(description="분해된 하위 질문들의 리스트")

def create_decomposition_chain():
    """
    Query Decomposition 체인 생성
    Returns:
        분해 체인 객체
    """
    # ChatPromptTemplate를 사용하여 시스템 메시지와 유저 메시지로 구성
    decomposition_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 복잡한 질문을 분석하는 전문가입니다. 주어진 질문을 원래 질문에 답하기 위해 필요한 하위 질문들로 분해해주세요.

분해 원칙:
1. 각 하위 질문은 독립적으로 답변 가능해야 함
2. 모든 하위 질문의 답변을 종합하면 원래 질문에 답할 수 있어야 함
3. 하위 질문은 구체적이고 명확해야 함
4. 시간적 순서나 논리적 순서를 고려
5. 3-6개의 하위 질문으로 분해

각 하위 질문과 함께 해당 질문이 필요한 이유도 제공해주세요."""),
        ("user", "원본 질문: {question}")
    ])
    
    # LLM 설정 및 구조화된 출력 적용
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3)
    
    # with_structured_output을 사용하여 Pydantic 모델로 구조화된 출력 생성
    structured_llm = llm.with_structured_output(DecompositionResult)
    
    # 체인 구성
    return decomposition_prompt | structured_llm

In [19]:
test_questions = [
    "리비안의 사업 모델이 테슬라와 어떻게 다르며, 향후 5년간 성장 가능성은?",
    "테슬라의 주요 경영진은 누구이며, 그들의 전략적 결정이 회사에 미친 영향은?",
]

# Query Decomposition 체인 생성
decomposition_chain = create_decomposition_chain()

for i, question in enumerate(test_questions, 1):
    print(f"\n[{i}] 원본 질문:")
    print(f"    {question}")
    
    # 질문 분해
    sub_questions = decomposition_chain.invoke({"question": question})

    print(f"\n    분해된 하위 질문 ({len(sub_questions.sub_queries)}개):")
    for j, sub_q in enumerate(sub_questions.sub_queries, 1):
        print(f"      {j}. {sub_q.sub_query} (이유: {sub_q.reasoning})")

    print("-" * 80)


[1] 원본 질문:
    리비안의 사업 모델이 테슬라와 어떻게 다르며, 향후 5년간 성장 가능성은?

    분해된 하위 질문 (5개):
      1. 리비안의 사업 모델은 무엇인가? (이유: 리비안의 사업 모델을 이해해야 테슬라와의 차이점을 비교할 수 있다.)
      2. 테슬라의 사업 모델은 무엇인가? (이유: 테슬라의 사업 모델을 알아야 리비안과의 차이점을 명확히 파악할 수 있다.)
      3. 리비안과 테슬라의 사업 모델은 어떤 점에서 다른가? (이유: 두 회사의 사업 모델 차이를 구체적으로 분석해야 원본 질문의 첫 부분에 답할 수 있다.)
      4. 리비안의 현재 시장 위치와 경쟁력은 어떠한가? (이유: 리비안의 성장 가능성을 평가하기 위해 현재 시장 위치와 경쟁력을 알아야 한다.)
      5. 향후 5년간 리비안의 성장 가능성에 영향을 미칠 주요 요인은 무엇인가? (이유: 성장 가능성을 예측하기 위해 영향을 미치는 주요 요인을 분석해야 한다.)
--------------------------------------------------------------------------------

[2] 원본 질문:
    테슬라의 주요 경영진은 누구이며, 그들의 전략적 결정이 회사에 미친 영향은?

    분해된 하위 질문 (3개):
      1. 테슬라의 주요 경영진은 누구인가? (이유: 테슬라의 주요 경영진을 알아야 그들이 회사에 어떤 영향을 미쳤는지 평가할 수 있기 때문이다.)
      2. 각 주요 경영진이 테슬라에서 내린 주요 전략적 결정은 무엇인가? (이유: 경영진의 전략적 결정을 파악해야 그 결정들이 회사에 어떤 영향을 미쳤는지 분석할 수 있기 때문이다.)
      3. 이러한 전략적 결정들이 테슬라의 경영 성과나 시장 위치에 어떤 영향을 미쳤는가? (이유: 전략적 결정이 회사에 미친 구체적인 영향을 이해하기 위해서는 경영 성과나 시장 위치 변화를 살펴봐야 하기 때문이다.)
------------------------

In [23]:
sub_questions.sub_queries[0].sub_query

'테슬라의 주요 경영진은 누구인가?'

#### 5.2.2 Decomposition 기반 검색기

In [24]:
def create_decomposition_retriever(vector_store):
    """
    분해 기반 검색기 생성
    
    Args:
        vector_store: 벡터 저장소
    
    Returns:
        분해 기반 검색기
    """
    from langchain_core.runnables import chain
    from collections import Counter
    
    # 분해 체인
    decomposition_chain = create_decomposition_chain()
    
    # 기본 검색기
    base_retriever = vector_store.as_retriever(search_kwargs={"k": 3})
    
    @chain
    def decomposed_retrieval(question):
        """분해된 질문들로 검색 수행"""
        # 1단계: 질문 분해
        sub_questions = decomposition_chain.invoke({"question": question})

        print(f"🧩 질문을 {len(sub_questions.sub_queries)}개로 분해:")
        for i, sub_q in enumerate(sub_questions.sub_queries, 1):
            print(f"  {i}. {sub_q.sub_query} (이유: {sub_q.reasoning})")
        print()
        
        # 2단계: 각 하위 질문으로 검색
        all_docs = []
        for i, sub_question in enumerate(sub_questions.sub_queries, 1):
            print(f"🔍 하위 질문 {i} 검색: {sub_question.sub_query}")

            sub_docs = base_retriever.invoke(sub_question.sub_query)
            all_docs.extend(sub_docs)
            
            print(f"   검색 결과: {len(sub_docs)}개 문서")
        
        # 3단계: 중복 제거 및 정렬
        # 문서 빈도 계산 (같은 문서가 여러 쿼리에서 나온 경우)
        doc_counter = Counter()
        doc_map = {}

        for doc in all_docs:
            # 문서 식별자 생성 (내용의 해시값 사용)
            doc_id = hash(doc.page_content[:200])
            doc_counter[doc_id] += 1
            if doc_id not in doc_map:
                doc_map[doc_id] = doc
        
        # 빈도 기반 정렬
        ranked_docs = [
            doc_map[doc_id] for doc_id, count in doc_counter.most_common()
        ]


        print(f"\n📋 최종 결과: {len(ranked_docs)}개 문서 (중복 제거 후)")

        return ranked_docs[:5], doc_counter  # 상위 5개 반환

    return decomposed_retrieval

In [25]:
# Decomposition 검색기 테스트

decomp_retriever = create_decomposition_retriever(vector_store)

complex_query = "리비안의 기술적 우위와 시장 전략을 테슬라와 비교해서 분석해주세요"

print(f"복잡한 질문: {complex_query}\n")

decomp_results, doc_counter = decomp_retriever.invoke(complex_query)

print("\n📊 검색 결과 요약:")
for i, doc in enumerate(decomp_results[:5], 1):
    print(f"[{i}] {doc.page_content[:80]}...")
    print(f"    출처: {doc.metadata.get('source', 'N/A')}")
    print(f"    문서 ID: {hash(doc.page_content[:200])}")
    print()

복잡한 질문: 리비안의 기술적 우위와 시장 전략을 테슬라와 비교해서 분석해주세요

🧩 질문을 5개로 분해:
  1. 리비안의 기술적 우위는 무엇인가요? (이유: 리비안의 기술적 강점을 파악해야 테슬라와 비교할 수 있습니다.)
  2. 리비안의 시장 전략은 무엇인가요? (이유: 리비안의 시장 접근 방식을 이해해야 테슬라와의 차별점을 분석할 수 있습니다.)
  3. 테슬라의 기술적 우위는 무엇인가요? (이유: 테슬라의 기술적 강점을 알아야 리비안과 비교할 수 있습니다.)
  4. 테슬라의 시장 전략은 무엇인가요? (이유: 테슬라의 시장 전략을 파악해야 리비안과의 차이를 분석할 수 있습니다.)
  5. 리비안과 테슬라의 기술적 우위와 시장 전략을 비교 분석하면 어떤 차이점과 유사점이 있나요? (이유: 각 회사의 기술과 시장 전략 정보를 종합하여 비교 분석함으로써 원본 질문에 답할 수 있습니다.)

🔍 하위 질문 1 검색: 리비안의 기술적 우위는 무엇인가요?
   검색 결과: 3개 문서
🔍 하위 질문 2 검색: 리비안의 시장 전략은 무엇인가요?
   검색 결과: 3개 문서
🔍 하위 질문 3 검색: 테슬라의 기술적 우위는 무엇인가요?
   검색 결과: 3개 문서
🔍 하위 질문 4 검색: 테슬라의 시장 전략은 무엇인가요?
   검색 결과: 3개 문서
🔍 하위 질문 5 검색: 리비안과 테슬라의 기술적 우위와 시장 전략을 비교 분석하면 어떤 차이점과 유사점이 있나요?
   검색 결과: 3개 문서

📋 최종 결과: 10개 문서 (중복 제거 후)

📊 검색 결과 요약:
[1] <Document>
## 비즈니스 전략

Tesla의 전략은 배터리 비용을 줄이기 위해 고가, 소량 차량으로 시작한 다음 더 저렴하고 대량 차량...
    출처: data\테슬라_KR.md
    문서 ID: -1158034330782906311

[2] <Document>
### 소프트웨어

Tesla는 무선 업데이트를 사용하여 강력한 온보드 컴퓨터를 통해 기능을 추가하거나 문제를 해결합니다.


In [26]:
# 문서 빈도 분석
print("📈 문서 빈도 분석:")
for doc_id, count in doc_counter.items():
    print(f"  문서 ID {doc_id}: {count}회")

📈 문서 빈도 분석:
  문서 ID -3360246965996940063: 1회
  문서 ID 4629480250970497807: 3회
  문서 ID -507681529169896070: 1회
  문서 ID 1102557081532371287: 1회
  문서 ID -1158034330782906311: 4회
  문서 ID -8290878284324221309: 1회
  문서 ID 3590293928235536470: 1회
  문서 ID 8495608624983104621: 1회
  문서 ID -7697455055500314248: 1회
  문서 ID -3935713560814163843: 1회



---

## 6. Step-Back Prompting

### 6.1 개념과 특징

Step-Back Prompting은 구체적인 질문에서 한 걸음 물러나 더 일반적이고 포괄적인 질문을 먼저 생성한 후, 이를 통해 배경 지식을 확보하고 원래 질문에 답하는 기법입니다.

#### 🎯 **핵심 특징**
- 일반적 맥락에서 구체적 답변으로 접근
- 배경 지식과 기본 개념 우선 확보
- 복잡한 질문에 대한 이해도 향상
- 두 단계 검색: Step-back + Original

<div align="center">
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/query_stepback.png" alt="Step-Back Prompting" width="700">
<br><i>Step-Back Prompting 프로세스</i>
</div>

### 6.2 구현

#### 6.2.1 Few-Shot Step-Back 생성기

In [27]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

def create_step_back_chain():
    """
    Step-Back 질문 생성 체인 생성
    
    Returns:
        Step-Back 체인 객체
    """
    # Few-Shot 예제 정의
    examples = [
        {
            "input": "리비안의 R1T 트럭의 배터리 용량은?",
            "output": "전기차 트럭의 일반적인 배터리 시스템과 성능 특성은 무엇인가?",
        },
        {
            "input": "테슬라 모델 S의 2024년 판매량은?",
            "output": "프리미엄 전기차 시장의 현재 동향과 주요 지표는 무엇인가?",
        },
        {
            "input": "애플의 M1 칩이 전력 효율성에서 인텔보다 우수한 이유는?",
            "output": "반도체 아키텍처가 전력 효율성에 미치는 핵심 요인들은 무엇인가?",
        },
        {
            "input": "아마존 AWS의 2023년 3분기 매출 성장률은?",
            "output": "클라우드 컴퓨팅 서비스 시장의 성장 패턴과 주요 동력은 무엇인가?",
        },
        {
            "input": "네이버의 AI 검색 알고리즘이 구글과 다른 점은?",
            "output": "검색 엔진의 AI 기술 적용 방식과 차별화 전략은 어떻게 구분되는가?",
        }
    ]
    
    # 예제 프롬프트 템플릿
    example_prompt = ChatPromptTemplate.from_messages([
        ("human", "{input}"),
        ("ai", "{output}"),
    ])
    
    # Few-Shot 프롬프트 생성
    few_shot_prompt = FewShotChatMessagePromptTemplate(
        example_prompt=example_prompt,
        examples=examples,
    )
    
    # 전체 프롬프트 구성
    step_back_prompt = ChatPromptTemplate.from_messages([
        (
            "system", 
            """당신은 질문 분석 전문가입니다. 구체적이고 세부적인 질문을 받았을 때, 
            해당 질문이 속한 더 넓은 개념이나 일반적인 원리에 대한 'Step-Back' 질문을 생성하는 것이 임무입니다.

            Step-Back 질문의 특징:
            1. 원래 질문보다 더 일반적이고 포괄적
            2. 해당 분야의 기본 개념이나 원리를 다룸
            3. 배경 지식 확보에 도움이 됨
            4. 원래 질문에 답하기 위한 맥락 제공

            다음은 좋은 Step-Back 질문의 예시들입니다:"""
        ),
        few_shot_prompt,
        ("user", "{question}"),
    ])
    
    # LLM 및 체인 구성
    llm = ChatOpenAI(model='gpt-4.1-mini', temperature=0.1)
    
    return step_back_prompt | llm | StrOutputParser()

In [28]:
step_back_chain = create_step_back_chain()

test_questions = [
    "리비안의 2024년 1분기 생산량은 몇 대인가요?",
    "테슬라 Model Y의 슈퍼차징 시간은?",
]

print("🔄 Step-Back 질문 생성 데모")
for i, question in enumerate(test_questions, 1):
    print(f"\n[{i}] 구체적 질문: {question}")
    
    step_back_question = step_back_chain.invoke({"question": question})
    print(f"    Step-Back 질문: {step_back_question}")
    
    print("-" * 60)


🔄 Step-Back 질문 생성 데모

[1] 구체적 질문: 리비안의 2024년 1분기 생산량은 몇 대인가요?
    Step-Back 질문: 전기차 제조업체의 생산량 결정 요인과 분기별 생산량 추세는 어떻게 나타나는가?
------------------------------------------------------------

[2] 구체적 질문: 테슬라 Model Y의 슈퍼차징 시간은?
    Step-Back 질문: 전기차 충전 기술의 기본 원리와 충전 시간에 영향을 미치는 주요 요소들은 무엇인가?
------------------------------------------------------------


#### 6.2.2 도메인별 Step-Back 생성기

In [29]:
def create_domain_step_back_chain(domain="technology"):
    """
    도메인별 Step-Back 체인 생성
    
    Args:
        domain (str): 도메인 분야
    
    Returns:
        도메인 특화 Step-Back 체인
    """
    domain_examples = {
        "technology": [
            {
                "input": "iPhone 15 Pro의 A17 Pro 칩의 트랜지스터 수는?",
                "output": "모바일 프로세서의 성능 향상을 위한 기술적 접근 방식과 트랜지스터 밀도의 역할은?"
            },
            {
                "input": "ChatGPT-4의 파라미터 수는 정확히 몇 개인가?",
                "output": "대규모 언어 모델의 성능과 파라미터 수 간의 관계는 어떻게 이해될 수 있는가?"
            }
        ],
        "business": [
            {
                "input": "2023년 네이버의 광고 매출은 얼마인가?",
                "output": "디지털 플랫폼 기업의 수익 구조와 광고 사업의 역할은 무엇인가?"
            },
            {
                "input": "삼성전자의 2024년 반도체 부문 투자 계획은?",
                "output": "글로벌 반도체 기업들의 투자 전략과 시장 경쟁력 확보 방안은?"
            }
        ],
        "finance": [
            {
                "input": "카카오뱅크의 2023년 순이익은 얼마인가?",
                "output": "디지털 은행의 수익성 모델과 전통 은행과의 차이점은 무엇인가?"
            },
            {
                "input": "비트코인의 현재 해시레이트는?",
                "output": "암호화폐의 네트워크 보안과 채굴 생태계의 원리는 어떻게 작동하는가?"
            }
        ]
    }
    
    examples = domain_examples.get(domain, domain_examples["technology"])
    
    example_prompt = ChatPromptTemplate.from_messages([
        ("human", "{input}"),
        ("ai", "{output}"),
    ])
    
    few_shot_prompt = FewShotChatMessagePromptTemplate(
        example_prompt=example_prompt,
        examples=examples,
    )
    
    domain_descriptions = {
        "technology": "기술 및 IT 분야의 전문가",
        "business": "비즈니스 및 경영 분야의 전문가", 
        "finance": "금융 및 투자 분야의 전문가"
    }
    
    step_back_prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            f"""당신은 {domain_descriptions.get(domain, '기술 분야의')}입니다. 
            구체적인 질문을 받았을 때, 해당 분야의 더 넓은 개념이나 원리에 대한 
            Step-Back 질문을 생성해주세요.
            
            다음은 {domain} 분야의 좋은 Step-Back 질문 예시들입니다:"""
        ),
        few_shot_prompt,
        ("user", "{question}"),
    ])
    
    llm = ChatOpenAI(model='gpt-4.1-mini', temperature=0.1)
    
    return step_back_prompt | llm | StrOutputParser()

In [30]:
domains = ["technology", "business", "finance"]
test_question = "테슬라의 2024년 3분기 매출은?"

print("🏢 도메인별 Step-Back Prompting")
print("=" * 80)
print(f"구체적 질문: {test_question}\n")

for domain in domains:
    chain = create_domain_step_back_chain(domain)
    step_back_q = chain.invoke({"question": test_question})
    
    print(f"[{domain.upper()}] {step_back_q}")
    print("-" * 60)

🏢 도메인별 Step-Back Prompting
구체적 질문: 테슬라의 2024년 3분기 매출은?

[TECHNOLOGY] 기업의 분기별 매출 변동이 장기적인 성장 전략에 미치는 영향은 무엇인가?
------------------------------------------------------------
[BUSINESS] 자동차 산업에서 매출 성장의 주요 동인과 기업의 장기적인 수익성 확보 전략은 무엇인가?
------------------------------------------------------------
[FINANCE] 테슬라의 매출 성장에 영향을 미치는 주요 산업 동향과 경쟁 요인은 무엇인가?
------------------------------------------------------------


#### 6.2.3 Step-Back 기반 검색기

In [31]:
def create_step_back_retriever(vector_store, domain="technology"):
    """
    Step-Back 기반 검색기 생성
    
    Args:
        vector_store: 벡터 저장소
        domain (str): 도메인 설정
    
    Returns:
        Step-Back 검색기
    """
    from langchain_core.runnables import chain
    
    # Step-Back 질문 생성 체인
    step_back_chain = create_domain_step_back_chain(domain)
    
    # 기본 검색기
    base_retriever = vector_store.as_retriever(search_kwargs={"k": 3})
    
    @chain
    def step_back_retrieval(question):
        """Step-Back 기반 검색 수행"""
        print(f"🔙 Step-Back Prompting 검색")
        print(f"원본 질문: {question}")
        
        # 1단계: Step-Back 질문 생성
        step_back_question = step_back_chain.invoke({"question": question})
        print(f"Step-Back 질문: {step_back_question}")
        
        # 2단계: 두 질문으로 각각 검색
        print("\n🔍 검색 수행:")
        
        # 원본 질문으로 검색
        print("  - 원본 질문으로 검색...")
        original_docs = base_retriever.invoke(question)
        
        # Step-Back 질문으로 검색
        print("  - Step-Back 질문으로 검색...")
        step_back_docs = base_retriever.invoke(step_back_question)
        
        # 3단계: 결과 통합
        all_docs = original_docs + step_back_docs
        
        # 중복 제거
        unique_docs = []
        seen_contents = set()
        
        for doc in all_docs:
            content_hash = hash(doc.page_content[:200])
            if content_hash not in seen_contents:
                seen_contents.add(content_hash)
                unique_docs.append(doc)
        
        print(f"\n📋 검색 결과:")
        print(f"  - 원본 검색: {len(original_docs)}개")
        print(f"  - Step-Back 검색: {len(step_back_docs)}개")
        print(f"  - 통합 결과: {len(unique_docs)}개 (중복 제거)")
        
        return unique_docs[:8]  # 상위 8개 반환
    
    return step_back_retrieval

In [32]:
# Step-Back 검색기 테스트
step_back_retriever = create_step_back_retriever(vector_store, domain="technology")

test_query = "리비안의 신규 모델 출시 계획은?"
results = step_back_retriever.invoke(test_query)

for i, doc in enumerate(results[:5], 1):
    print(f"[{i}] {doc.page_content[:100]}...")
    print(f"    출처: {doc.metadata.get('source', 'N/A')}")
    print("-" * 50)

🔙 Step-Back Prompting 검색
원본 질문: 리비안의 신규 모델 출시 계획은?
Step-Back 질문: 전기차 시장에서 신모델 출시가 기업의 경쟁력과 시장 점유율에 미치는 영향은 무엇인가?

🔍 검색 수행:
  - 원본 질문으로 검색...
  - Step-Back 질문으로 검색...

📋 검색 결과:
  - 원본 검색: 3개
  - Step-Back 검색: 3개
  - 통합 결과: 6개 (중복 제거)
[1] <Document>
- 2022년 10월, 느슨한 토크 볼트로 인해 13,000대의 차량을 자발적으로 리콜.
- 2023년 6월, Tesla의 북미 충전 시스템(NACS) 채택을 ...
    출처: data\리비안_KR.md
--------------------------------------------------
[2] <Document>
**역사**

**초창기 (2009–15):**

- 2009년 R. J. 스캐린지가 Mainstream Motors로 설립.
- 2011년 Rivian Aut...
    출처: data\리비안_KR.md
--------------------------------------------------
[3] <Document>
- **Model S:** 리프트백 차체 스타일과 듀얼 모터, 전륜 구동 레이아웃을 갖춘 풀사이즈 고급차. Model S 개발은 2007년 이전에 시작되었으며 ...
    출처: data\테슬라_KR.md
--------------------------------------------------
[4] <Document>
Tesla의 차량 생산은 2008년 Roadster로 시작하여 Model S (2012), Model X (2015), Model 3 (2017), Model ...
    출처: data\테슬라_KR.md
--------------------------------------------------
[5] <Document>
Roadster

In [33]:
def create_step_back_rag_chain(vector_store, domain="technology"):
    """
    Step-Back RAG 체인 생성 (검색 + 답변 생성)
    
    Args:
        vector_store: 벡터 저장소
        domain (str): 도메인 설정
    
    Returns:
        완전한 Step-Back RAG 체인
    """
    # Step-Back 검색기
    step_back_retriever = create_step_back_retriever(vector_store, domain)
    
    # 답변 생성 프롬프트
    answer_prompt = ChatPromptTemplate.from_template(
        """당신은 {domain} 분야의 전문가입니다. 주어진 컨텍스트를 바탕으로 질문에 정확하고 포괄적으로 답변해주세요.

컨텍스트:
{context}

질문: {question}

답변 원칙:
1. 제공된 컨텍스트 정보를 우선적으로 활용
2. 배경 지식과 구체적 정보를 적절히 조합
3. 불확실한 정보는 명시적으로 표시
4. 답변의 근거를 명확히 제시

답변:"""
    )
    
    def format_docs(docs):
        """문서 포맷팅"""
        return "\n\n".join([
            f"[문서 {i+1}] {doc.page_content}"
            for i, doc in enumerate(docs)
        ])
    
    # LLM 초기화
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.1)
    
    # 전체 체인 구성
    rag_chain = (
        {
            "context": step_back_retriever | format_docs,
            "question": RunnablePassthrough(),
            "domain": lambda _: domain
        }
        | answer_prompt
        | llm
        | StrOutputParser()
    )
    
    return rag_chain

In [34]:
# Step-Back RAG 테스트

step_back_rag = create_step_back_rag_chain(vector_store, "business")
answer = step_back_rag.invoke(test_query)

print(f"질문: {test_query}")
print(f"답변: {answer}")

🔙 Step-Back Prompting 검색
원본 질문: 리비안의 신규 모델 출시 계획은?
Step-Back 질문: 전기차 시장에서 신제품 출시가 기업의 성장과 시장 점유율 확대에 미치는 영향은 무엇인가?

🔍 검색 수행:
  - 원본 질문으로 검색...
  - Step-Back 질문으로 검색...

📋 검색 결과:
  - 원본 검색: 3개
  - Step-Back 검색: 3개
  - 통합 결과: 6개 (중복 제거)
질문: 리비안의 신규 모델 출시 계획은?
답변: 리비안(Rivian)의 신규 모델 출시 계획에 대해 제공된 컨텍스트를 바탕으로 정리하면 다음과 같습니다.

1. **신규 모델 공개 시점 및 종류**  
   - 2024년 3월에 리비안은 더 작은 가격대의 R2 SUV 및 R3 모델을 공개했습니다.  
   - 이는 기존의 R1T(픽업 트럭)와 R1S(SUV)보다 상대적으로 저렴하고 소형화된 모델로 보이며, 리비안의 제품 라인업 확장 및 대중화 전략의 일환으로 해석됩니다.  
   - (출처: 문서 1)

2. **기존 모델과의 관계 및 시장 전략**  
   - R1T와 R1S는 2021년부터 배송이 시작된 리비안의 첫 모델로, 프리미엄 전기 픽업 및 SUV 시장을 겨냥했습니다.  
   - R2, R3 모델은 가격대를 낮추고 소형 SUV 시장에 진입함으로써 보다 폭넓은 고객층을 확보하려는 전략으로 보입니다.  
   - (출처: 문서 1, 문서 5)

3. **추가적인 출시 계획에 대한 정보**  
   - 현재 제공된 문서 내에서는 R2, R3 외에 다른 신규 모델 출시 계획에 대한 구체적인 언급은 없습니다.  
   - 따라서 2024년 이후의 추가 모델 출시 계획이나 세부 사양 등은 불확실하며, 공식 발표가 필요합니다.

---

**요약:**  
리비안은 2024년 3월에 더 작은 가격대의 R2 SUV 및 R3 모델을 공개하여 기존의 프리미엄 R1T, R1S 라인업을 보완하고 있습니다. 이 두 모델은 리비안의 제품 포트폴리오 확장과 대중화


---

## 7. HyDE (Hypothetical Document Embedding)

### 7.1 개념과 특징

HyDE는 주어진 질문에 대해 가상의 이상적인 답변 문서를 LLM으로 생성한 후, 이 가상 문서의 임베딩을 사용하여 실제 문서를 검색하는 기법입니다.

#### 🎯 **핵심 특징**
- 질문-문서 임베딩 공간의 불일치 해결
- 가상 답변을 통한 의미적 정렬 개선
- Zero-shot 검색 성능 향상
- 질문보다 문서와 유사한 형태로 검색

<div align="center">
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/query_HyDE.png" alt="HyDE" width="700">
<br><i>HyDE (Hypothetical Document Embedding) 프로세스</i>
</div>

### 7.2 구현

#### 7.2.1 기본 HyDE 구현

In [35]:
def create_hyde_chain():
    """
    기본 HyDE 체인 생성
    Returns:
        HyDE 체인 객체
    """
    
    # HyDE 프롬프트 템플릿 정의
    # HyDE는 "Hypothetical Document Engineering"의 약자로, 이상적인 답변 문서를 생성하는 데 중점을 둡니다.
    # 이 프롬프트는 주어진 질문에 대해 이상적인 답변 문서를 작성하도록 LLM을 안내합니다.
    hyde_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 전문적인 문서 작성자입니다. 주어진 질문에 대한 이상적인 답변 문서를 작성해주세요.

작성 원칙:
1. 질문에 직접적으로 답하는 내용 포함
2. 관련된 배경 정보와 상세 설명 추가
3. 전문적이고 신뢰할 수 있는 톤 사용
4. 구체적인 사실과 데이터 포함 (가능한 경우)
5. 문서는 200-400단어 길이로 작성

이상적인 답변 문서만 작성하세요."""),
        ("user", "질문: {question}")
    ])
    
    # LLM 및 파서 설정
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3)

    return hyde_prompt | llm | StrOutputParser()

In [36]:
# HyDE 체인 생성
hyde_chain = create_hyde_chain()

test_questions = [
    "리비안의 핵심 기술 우위는 무엇인가요?",
    "테슬라의 주요 경영진은 누구인가요?",
]


for i, question in enumerate(test_questions, 1):
    print(f"\n[{i}] 질문: {question}")
    print("-" * 50)
    
    hypothetical_doc = hyde_chain.invoke({"question": question})
    print(f"가상 문서:\n{hypothetical_doc}")
    
    print("=" * 80)


[1] 질문: 리비안의 핵심 기술 우위는 무엇인가요?
--------------------------------------------------
가상 문서:
리비안(Rivian)은 전기차(EV) 시장에서 빠르게 주목받는 미국의 전기차 제조업체로, 특히 전기 픽업트럭과 SUV 분야에서 강력한 입지를 구축하고 있습니다. 리비안의 핵심 기술 우위는 다음과 같은 주요 요소들로 요약할 수 있습니다.

첫째, **스케이트보드 플랫폼(스케이트보드 아키텍처)**입니다. 리비안은 배터리 팩, 전기 모터, 서스펜션, 제어 시스템 등을 하나의 평평한 섀시에 통합한 스케이트보드 플랫폼을 개발했습니다. 이 구조는 차량 설계의 유연성을 극대화하며, 다양한 차종에 동일한 플랫폼을 적용할 수 있어 생산 효율성과 비용 절감에 크게 기여합니다. 또한, 배터리 팩을 차체 하단에 배치해 무게 중심을 낮추고 주행 안정성을 높였습니다.

둘째, **네 개의 독립 전기 모터(쿼드 모터 시스템)**입니다. 리비안은 각 바퀴에 독립적으로 전력을 공급하는 쿼드 모터 구동 방식을 채택해 뛰어난 오프로드 성능과 정밀한 토크 벡터링을 구현합니다. 이를 통해 험로 주행 시 바퀴별로 최적의 동력을 분배하여 접지력을 극대화하고, 고속 주행 시에도 안정적인 핸들링을 제공합니다.

셋째, **고성능 배터리 기술과 장거리 주행 능력**입니다. 리비안은 105kWh, 135kWh, 180kWh 용량의 배터리 팩을 제공하며, 최대 640km(400마일) 이상의 주행 거리를 확보했습니다. 이는 경쟁사 대비 우수한 수준으로, 장거리 여행과 상용 운송에 적합한 전기차로 평가받고 있습니다.

넷째, **첨단 소프트웨어 및 OTA(Over-The-Air) 업데이트** 기능입니다. 리비안은 차량의 성능 최적화, 사용자 경험 개선, 보안 강화 등을 위해 정기적인 무선 소프트웨어 업데이트를 지원하며, 이를 통해 지속적으로 차량 기능을 향상시키고 있습니다.

마지막으로, **친환경적이고 지속 가능한 제조 공정**에 대한 집중도 리비안의 차

#### 7.2.2 도메인별 HyDE 구현

In [37]:
def create_domain_hyde_chain(domain="technology"):
    """
    도메인별 HyDE 체인 생성
    Args:
        domain (str): 도메인 분야
    Returns:
        도메인 특화 HyDE 체인
    """
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_openai import ChatOpenAI
    from langchain_core.output_parsers import StrOutputParser
    
    # 도메인별 시스템 메시지 정의
    domain_system_messages = {
        "technology": """당신은 기술 분야의 전문 연구원입니다. 주어진 기술 질문에 대한 전문적인 기술 문서를 작성해주세요.

기술 문서 작성 원칙:
1. 기술적 정확성과 구체성 유지
2. 관련 기술 스택과 아키텍처 언급
3. 성능 지표와 벤치마크 포함
4. 기술의 장단점 객관적 분석
5. 실제 구현 사례와 응용 분야 제시

전문 기술 문서만 작성하세요.""",
        
        "business": """당신은 비즈니스 전략 컨설턴트입니다. 주어진 비즈니스 질문에 대한 전문적인 분석 보고서를 작성해주세요.

비즈니스 보고서 작성 원칙:
1. 시장 동향과 경쟁 환경 분석
2. 재무적 지표와 성과 측정 포함
3. 전략적 시사점과 추천 사항 제시
4. 리스크 요인과 기회 요소 분석
5. 데이터 기반의 객관적 결론 도출

비즈니스 분석 보고서만 작성하세요.""",
        
        "academic": """당신은 학술 연구자입니다. 주어진 연구 질문에 대한 학술적 문헌 요약을 작성해주세요.

학술 문서 작성 원칙:
1. 이론적 배경과 선행 연구 검토
2. 연구 방법론과 실증 분석 포함
3. 객관적이고 중립적인 관점 유지
4. 인용과 참고문헌 스타일 적용
5. 연구의 한계점과 향후 연구 방향 제시

학술 문헌 요약만 작성하세요."""
    }
    
    # 선택된 도메인의 시스템 메시지 가져오기 (기본값: technology)
    system_message = domain_system_messages.get(domain, domain_system_messages["technology"])
    
    # ChatPromptTemplate를 사용하여 시스템 메시지와 유저 메시지로 구성
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("user", "질문: {question}")
    ])
    
    # LLM 및 파서 설정
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3)

    return prompt | llm | StrOutputParser()

In [38]:
# 도메인별 HyDE 체인 테스트
test_question = "전기차 회사들의 경쟁 전략은?"
domains = ["technology", "business", "academic"]

print("🏢 도메인별 HyDE 비교")
print("=" * 80)
print(f"질문: {test_question}\n")

for domain in domains:
    hyde_chain = create_domain_hyde_chain(domain)
    doc = hyde_chain.invoke({"question": test_question})

    print(f"[{domain.upper()}] 가상 문서:")
    print(doc[:300] + "..." if len(doc) > 300 else doc)
    print("-" * 60)


🏢 도메인별 HyDE 비교
질문: 전기차 회사들의 경쟁 전략은?

[TECHNOLOGY] 가상 문서:
# 전기차(EV) 회사들의 경쟁 전략 분석

## 1. 서론
전기차(Electric Vehicle, EV) 산업은 친환경 정책 강화, 배터리 기술 발전, 소비자 인식 변화에 힘입어 급성장하고 있다. 이에 따라 주요 전기차 제조사들은 기술 혁신, 생산 효율성, 브랜드 차별화, 생태계 구축 등 다양한 경쟁 전략을 구사하며 시장 점유율 확대에 나서고 있다. 본 문서에서는 전기차 회사들의 주요 경쟁 전략을 기술적, 경영적 측면에서 분석하고, 관련 기술 스택과 아키텍처, 성능 지표, 장단점, 실제 사례를 종합적으로 고찰한다.

## 2. 주...
------------------------------------------------------------
[BUSINESS] 가상 문서:
# 전기차(EV) 회사들의 경쟁 전략 분석 보고서

## 1. 시장 동향 및 경쟁 환경 분석

### 1.1 시장 동향
- **급성장하는 전기차 시장**: 글로벌 전기차 시장은 2020년대 들어 연평균 20~30% 이상의 성장률을 기록하며 내연기관차를 빠르게 대체 중임.
- **정부 정책 및 규제 강화**: 주요 국가들이 탄소 배출 규제를 강화하고 전기차 보조금 및 인프라 투자를 확대하여 시장 성장을 촉진.
- **배터리 기술 혁신**: 배터리 에너지 밀도 향상과 가격 하락이 전기차 가격 경쟁력 강화에 기여.
- **소프트웨어 및 ...
------------------------------------------------------------
[ACADEMIC] 가상 문서:
전기차(Electric Vehicle, EV) 산업에서 기업들의 경쟁 전략은 기술 혁신, 생산 효율성, 시장 세분화, 그리고 생태계 구축을 중심으로 전개되고 있다. 본 요약에서는 이론적 배경과 선행 연구를 토대로 전기차 회사들의 경쟁 전략을 분석하고, 주요 연구 방법론과 실증 결과를 검토하며, 향후 연구 방향을 제시

#### 7.2.3 HyDE 기반 검색기

In [39]:
def create_hyde_retriever(vector_store, domain="technology"):
    """
    HyDE 기반 검색기 생성
    
    Args:
        vector_store: 벡터 저장소
        domain (str): 도메인 설정
    
    Returns:
        HyDE 검색기
    """
    from langchain_core.runnables import chain
    
    # HyDE 체인
    hyde_chain = create_domain_hyde_chain(domain)
    
    # 기본 검색기
    base_retriever = vector_store.as_retriever(search_kwargs={"k": 5})
    
    @chain
    def hyde_retrieval(question):
        """HyDE 기반 검색 수행"""
        print(f"🧠 HyDE 검색 프로세스")
        print(f"원본 질문: {question}")
        
        # 1단계: 가상 문서 생성
        print("\n🔄 1단계: 가상 문서 생성")
        hypothetical_doc = hyde_chain.invoke({"question": question})
        print(f"가상 문서 길이: {len(hypothetical_doc)}자")
        print(f"가상 문서 미리보기: {hypothetical_doc[:150]}...")
        
        # 2단계: 가상 문서로 검색
        print("\n🔍 2단계: 가상 문서로 유사도 검색")
        retrieved_docs = base_retriever.invoke(hypothetical_doc)
        
        print(f"검색 결과: {len(retrieved_docs)}개 문서")
        
        return retrieved_docs
    
    return hyde_retrieval

In [40]:
hyde_retriever = create_hyde_retriever(vector_store, "business")

test_query = "리비안의 시장 경쟁력은?"
hyde_results = hyde_retriever.invoke(test_query)

print(f"\n📋 HyDE 검색 결과:")
for i, doc in enumerate(hyde_results, 1):
    print(f"[{i}] {doc.page_content[:100]}...")
    print(f"    출처: {doc.metadata.get('source', 'N/A')}")
    print("-" * 50)

🧠 HyDE 검색 프로세스
원본 질문: 리비안의 시장 경쟁력은?

🔄 1단계: 가상 문서 생성
가상 문서 길이: 1446자
가상 문서 미리보기: 리비안(Rivian)의 시장 경쟁력 분석 보고서

1. 시장 동향과 경쟁 환경 분석  
전기차(EV) 시장은 글로벌 친환경 정책 강화와 소비자 인식 변화에 힘입어 빠르게 성장하고 있습니다. 2024년 기준, 전기 픽업트럭 및 SUV 세그먼트는 아직 초기 단계이나, 포드...

🔍 2단계: 가상 문서로 유사도 검색
검색 결과: 5개 문서

📋 HyDE 검색 결과:
[1] <Document>
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J...
    출처: data\리비안_KR.md
--------------------------------------------------
[2] <Document>
- R1T 배송은 2021년 9월에 시작되어 Rivian은 완전 전기 픽업을 소비자 시장에 출시한 최초의 자동차 제조업체가 됨.
- 2021년 11월, Rivi...
    출처: data\리비안_KR.md
--------------------------------------------------
[3] <Document>
- 2022년 10월, 느슨한 토크 볼트로 인해 13,000대의 차량을 자발적으로 리콜.
- 2023년 6월, Tesla의 북미 충전 시스템(NACS) 채택을 ...
    출처: data\리비안_KR.md
--------------------------------------------------
[4] <Document>
**Volkswagen과의 파트너십 (2024)**

- 2024년 6월, Volkswagen Group은 전기 아키텍처 및 소프트웨어 기술 개발을 목표로 Ri...
    출처: data\리비안_KR.md
----------------------

#### 7.2.4 HyDE RAG 체인

In [41]:
def create_hyde_rag_chain(vector_store, domain="technology"):
    """
    완전한 HyDE RAG 체인 생성
    Args:
        vector_store: 벡터 저장소
        domain (str): 도메인 설정
    Returns:
        HyDE RAG 체인
    """
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_openai import ChatOpenAI
    from langchain_core.output_parsers import StrOutputParser
    from langchain_core.runnables import RunnablePassthrough
    
    # HyDE 검색기
    hyde_retriever = create_hyde_retriever(vector_store, domain)
    
    # 도메인별 시스템 메시지 정의
    domain_system_messages = {
        "technology": f"""당신은 {domain} 분야의 전문가입니다. HyDE 기법으로 검색된 기술 문서들을 바탕으로 질문에 정확하고 전문적으로 답변해주세요.

답변 원칙:
1. 검색된 문서의 정보를 우선적으로 활용
2. 기술적 정확성과 구체성을 유지
3. 불확실한 정보는 명시적으로 표시
4. 가능한 경우 구체적인 데이터나 사례 인용
5. 답변의 근거와 출처 명확히 제시
6. 기술적 세부사항과 실용적 관점 제공

전문가 답변만 제공하세요.""",
        
        "business": f"""당신은 {domain} 분야의 전문가입니다. HyDE 기법으로 검색된 비즈니스 문서들을 바탕으로 질문에 정확하고 전문적으로 답변해주세요.

답변 원칙:
1. 검색된 문서의 정보를 우선적으로 활용
2. 비즈니스 전략과 시장 관점에서 분석
3. 불확실한 정보는 명시적으로 표시
4. 가능한 경우 구체적인 데이터나 사례 인용
5. 답변의 근거와 출처 명확히 제시
6. 실행 가능한 인사이트와 추천사항 제공

전문가 답변만 제공하세요.""",
        
        "academic": f"""당신은 {domain} 분야의 전문가입니다. HyDE 기법으로 검색된 학술 문서들을 바탕으로 질문에 정확하고 전문적으로 답변해주세요.

답변 원칙:
1. 검색된 문서의 정보를 우선적으로 활용
2. 학술적 엄밀성과 객관성 유지
3. 불확실한 정보는 명시적으로 표시
4. 가능한 경우 구체적인 데이터나 사례 인용
5. 답변의 근거와 출처 명확히 제시
6. 이론적 배경과 실증적 증거 균형있게 제시

전문가 답변만 제공하세요."""
    }
    
    # 선택된 도메인의 시스템 메시지 (기본값: technology)
    system_message = domain_system_messages.get(domain, domain_system_messages["technology"])
    
    # ChatPromptTemplate를 사용하여 시스템 메시지와 유저 메시지로 구성
    answer_prompt = ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("user", """검색된 컨텍스트:
{context}

원본 질문: {question}""")
    ])
    
    def format_docs(docs):
        """문서 포맷팅"""
        formatted_docs = []
        for i, doc in enumerate(docs, 1):
            source = doc.metadata.get('source', 'Unknown')
            content = doc.page_content
            formatted_docs.append(f"[문서 {i}] (출처: {source})\n{content}")
        return "\n\n".join(formatted_docs)
    
    # LLM 초기화
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.1)
    
    # 전체 RAG 체인 구성
    hyde_rag_chain = (
        {
            "context": hyde_retriever | format_docs,
            "question": RunnablePassthrough(),
        }
        | answer_prompt
        | llm
        | StrOutputParser()
    )
    
    return hyde_rag_chain

In [42]:
# HyDE RAG 테스트

hyde_rag = create_hyde_rag_chain(vector_store, "business")

test_questions = [
    "리비안의 핵심 경쟁력은 무엇인가요?",
    "전기차 시장에서 테슬라와 리비안의 차이점은?",
]

for i, question in enumerate(test_questions, 1):
    print(f"\n[{i}] 질문: {question}")
    print("-" * 60)
    
    answer = hyde_rag.invoke(question)
    print(f"HyDE RAG 답변:\n{answer}")
    
    print("=" * 80)


[1] 질문: 리비안의 핵심 경쟁력은 무엇인가요?
------------------------------------------------------------
🧠 HyDE 검색 프로세스
원본 질문: 리비안의 핵심 경쟁력은 무엇인가요?

🔄 1단계: 가상 문서 생성
가상 문서 길이: 1504자
가상 문서 미리보기: 리비안(Rivian)의 핵심 경쟁력 분석 보고서

1. 시장 동향과 경쟁 환경 분석  
전기차(EV) 시장은 글로벌 친환경 정책 강화와 소비자 인식 변화에 힘입어 급성장 중입니다. 특히 픽업트럭과 SUV 세그먼트에서 전기차 수요가 증가하고 있으며, 아마존, 포드, GM...

🔍 2단계: 가상 문서로 유사도 검색
검색 결과: 5개 문서
HyDE RAG 답변:
리비안(Rivian)의 핵심 경쟁력은 다음과 같이 요약할 수 있습니다.

1. **전기차 시장에서의 선도적 제품 포트폴리오**  
   - 리비안은 2021년 9월 R1T 전기 픽업 트럭을 출시하며 완전 전기 픽업을 소비자 시장에 처음 선보인 선구자적 위치에 있습니다.  
   - R1T 픽업과 R1S SUV, 그리고 Amazon 전용 Electric Delivery Van(EDV) 등 다양한 전기차 라인업을 보유하고 있으며, 2026년에는 더 작고 저렴한 R2 SUV와 R3 소형 SUV 출시를 계획 중입니다.  
   - 이러한 제품 다양성은 북미 전기차 시장에서 차별화된 고객층 확보에 유리합니다. (문서 2, 5)

2. **강력한 투자 및 전략적 파트너십**  
   - 2021년 IPO를 통해 135억 달러를 조달했고, 2024년에는 Volkswagen Group이 전기 아키텍처 및 소프트웨어 기술 개발을 위해 최대 50억 달러를 투자할 의향을 밝혔습니다.  
   - Amazon, T. Rowe Price, Vanguard, BlackRock, Fidelity 등 대형 기관투자자가 최대 주주로 참여하고 있어 재무적 안정성과 성장 가능성을 뒷받침합니다.  
   - Amazon과의 협력으

---

## 8. 실습 문제 ~ 15:55분까지

### 🧪 실습 1: 기본 쿼리 확장 비교

**목표**: 다양한 쿼리 확장 기법을 구현하고 동일한 질문에 대한 결과를 비교해보세요.

**과제**:
1. 주어진 질문에 대해 모든 쿼리 확장 기법을 적용하세요.
2. 각 기법의 검색 결과를 분석하고 비교하세요.
3. 어떤 기법이 가장 적합한지 평가하세요.


In [ ]:
def solve_exercise_1():
    """실습 1: 기본 쿼리 확장 비교"""
    
    # 테스트할 질문
    test_question = "전기차 회사들의 혁신적인 기술 개발 현황은?"
    
    # TODO: 다음 기법들을 구현하고 비교하세요
    # 1. 기본 검색
    # 2. Query Reformulation
    # 3. Multi Query (3개 변형 질문)
    # 4. Decomposition (하위 질문 분해)
    # 5. Step-Back Prompting
    # 6. HyDE
    
    results = {}
    
    # 각 기법별 구현
    print("🔬 실습 1: 쿼리 확장 기법 비교")
    print("=" * 60)
    print(f"테스트 질문: {test_question}\n")
    
    # 여기에 코드 작성
    
    return results

# solve_exercise_1()

### 🧪 실습 2: 도메인별 성능 최적화

**목표**: 특정 도메인에 최적화된 쿼리 확장 시스템을 구축하세요.

**과제**:
1. 기술, 비즈니스, 학술 중 하나의 도메인을 선택하세요.
2. 해당 도메인에 특화된 프롬프트와 체인을 설계하세요.

In [ ]:
def solve_exercise_2(domain="technology"):
    """실습 2: 도메인별 성능 최적화"""
    
    # TODO: 선택한 도메인에 최적화된 시스템 구축
    # 1. 도메인별 전문 용어 사전 구축
    # 2. 도메인 특화 프롬프트 템플릿 작성
    
    domain_system = {
        "reformulation_chain": None,
        "multi_query_chain": None, 
        "hyde_chain": None,
    }
    
    print(f"🏢 실습 2: {domain} 도메인 최적화")
    print("=" * 60)
    
    # 여기에 코드 작성
    
    return domain_system

# solve_exercise_2("business")

---

### 🎯 **언제 어떤 기법을 사용할까?**

| 상황 | 추천 기법 | 이유 |
|------|-----------|------|
| 모호한 질문 | Multi Query | 다양한 해석으로 포괄적 검색 |
| 복잡한 질문 | Decomposition | 체계적 분해로 정확성 향상 |
| 전문적 질문 | Step-Back + HyDE | 배경지식 + 의미적 정렬 |
| 일반적 질문 | Reformulation | 간단하고 효과적 |
| Zero-shot 상황 | HyDE | 사전 지식 없이도 효과적 |

### 🚀 **성능 최적화 팁**

1. **도메인 특화**: 각 분야에 맞는 프롬프트 최적화
2. **하이브리드 접근**: 여러 기법의 장점 결합
3. **적응적 선택**: 질문 특성에 따른 동적 기법 선택
4. **캐싱 활용**: 비용과 지연시간 최적화
5. **지속적 평가**: 실제 사용 데이터로 성능 모니터링

### 📚 논문
- **HyDE**: "Precise Zero-Shot Dense Retrieval without Relevance Labels" (Gao et al., 2022)
- **Query2Doc**: "Query Expansion with Large Language Models" (Wang et al., 2023)
- **Step-Back Prompting**: "Take a Step Back: Evoking Reasoning via Abstraction" (Zheng et al., 2023)
- **Least-to-Most Prompting**: "Least-to-Most Prompting Enables Complex Reasoning" (Zhou et al., 2022)


---


```python

```

`(1) 가상 문서 생성`

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# HyDE를 위한 프롬프트 템플릿 생성
template = """주어진 질문에 대한 이상적인 문서 내용을 생성해주세요.
문서는 학술적이고 전문적인 톤으로 작성되어야 합니다.

질문: {question}

문서 내용:"""

hyde_prompt = ChatPromptTemplate.from_template(template)

# LLM 모델 초기화
hyde_llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 문서 생성 체인 생성
hyde_chain = hyde_prompt | hyde_llm | StrOutputParser()

# 문서 생성 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
hypothetical_doc = hyde_chain.invoke({"question": query})

print(f"쿼리: {query}")
print(f"문서 내용: {hypothetical_doc}")

`(2) 유사 문서 검색`

In [ ]:
# 가상 문서를 기반으로 실제 문서 검색
    
retrieved_docs = chroma_k_retriever.invoke(hypothetical_doc)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

`(3) 최종 답변 생성`

In [ ]:
# 최종 RAG를 위한 프롬프트 템플릿 생성
template = """다음 컨텍스트를 바탕으로 질문에 답변해주세요:

컨텍스트:
{context}

질문: {question}

답변:"""

rag_prompt =  ChatPromptTemplate.from_template(template)

# RAG 체인 생성
rag_llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
rag_chain = rag_prompt | rag_llm | StrOutputParser()
    
# RAG 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
context = format_docs(retrieved_docs)

answer = rag_chain.invoke({"context": context, "question": query})

print(f"쿼리: {query}")
print(f"답변: {answer}")

`(4) HyDE 체인 종합`

In [ ]:
# Step 1. 가상 문서 생성
query = "테슬라의 경영진을 분석해주세요."

hypothetical_doc = hyde_chain.invoke({"question": query})

# Step 2. 유사 문서 검색
retrieved_docs = chroma_k_retriever.invoke(hypothetical_doc)

# Step 3. 최종 답변 생성
final_answer = rag_chain.invoke(
    {
        "context": format_docs(retrieved_docs), 
        "question": query
    }
)

print(f"쿼리: {query}")
print(f"가상 문서 내용: {hypothetical_doc}")
print(f"답변: {final_answer}")